In [1]:
import torch
import model

In [2]:
model = model.ConvNet()
model.load_state_dict(torch.load("model.pkl", map_location='cpu'))

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [3]:
for name,parameters in model.named_parameters():
    print(name,':',parameters.size())

features.0.weight : torch.Size([16, 1, 3, 3])
features.0.bias : torch.Size([16])
features.2.weight : torch.Size([32, 16, 3, 3])
features.2.bias : torch.Size([32])
features.5.weight : torch.Size([32, 32, 3, 3])
features.5.bias : torch.Size([32])
classifier.0.weight : torch.Size([128, 6272])
classifier.0.bias : torch.Size([128])
classifier.2.weight : torch.Size([10, 128])
classifier.2.bias : torch.Size([10])


In [4]:
for name, sub_module in model.named_modules():
#     print(name, ':', sub_module)
    if type(sub_module) is torch.nn.Conv2d:
        print(name)
#         print(sub_module.in_channels)
#         print(sub_module.padding)
#         print(sub_module.stride)
        print(sub_module.kernel_size[0])
        print(type(sub_module.bias))

features.0
3
<class 'torch.nn.parameter.Parameter'>
features.2
3
<class 'torch.nn.parameter.Parameter'>
features.5
3
<class 'torch.nn.parameter.Parameter'>


In [5]:
def write_define(define_name, define_value, config_file):
    config_file.write("#define "+str(define_name)+" "+str(define_value)+"\n")
    
def generateConfig(model, in_row, in_col, path_to_config):
    config_file = open(path_to_config, "w")
    cov_cnt = 0
    pool_cnt = 0
    linear_cnt = 0
    in_ch = 0
    for name, sub_module in model.named_modules():
        if type(sub_module) is torch.nn.Conv2d:
            write_define("COV_" + str(cov_cnt) + "_IN_CH", sub_module.in_channels, config_file)
            write_define("COV_" + str(cov_cnt) + "_IN_ROW", in_row, config_file)
            write_define("COV_" + str(cov_cnt) + "_IN_COL", in_col, config_file)
            
            in_row = (in_row + 2 * sub_module.padding[0] - sub_module.kernel_size[0]) // sub_module.stride[0] + 1
            in_col = (in_col + 2 * sub_module.padding[0] - sub_module.kernel_size[0]) // sub_module.stride[0] + 1
            in_ch = sub_module.out_channels
            write_define("COV_" + str(cov_cnt) + "_OUT_CH", sub_module.out_channels, config_file)
            write_define("COV_" + str(cov_cnt) + "_OUT_ROW", in_row, config_file)
            write_define("COV_" + str(cov_cnt) + "_OUT_COL", in_col, config_file)
            
           
            write_define("COV_" + str(cov_cnt) + "_K", sub_module.kernel_size[0], config_file)
            write_define("COV_" + str(cov_cnt) + "_S", sub_module.stride[0], config_file)
            write_define("COV_" + str(cov_cnt) + "_P", sub_module.padding[0], config_file)
            if type(sub_module.bias) != None:
                write_define("COV_" + str(cov_cnt) + "_B", 1, config_file)
            else:
                write_define("COV_" + str(cov_cnt) + "_B", 0, config_file)
            cov_cnt = cov_cnt + 1
            config_file.write("\n")
        elif type(sub_module) is torch.nn.MaxPool2d:
            write_define("POOL_" + str(pool_cnt) + "_IN_CH", in_ch, config_file)
            write_define("POOL_" + str(pool_cnt) + "_IN_ROW", in_row, config_file)
            write_define("POOL_" + str(pool_cnt) + "_IN_COL", in_col, config_file)
            write_define("POOL_" + str(pool_cnt) + "_IN_PO", sub_module.kernel_size, config_file)
            in_row = in_row // sub_module.kernel_size
            in_col = in_col // sub_module.kernel_size
            pool_cnt = pool_cnt + 1
            config_file.write("\n")
        elif type(sub_module) is torch.nn.Linear:
            write_define("LINEAR_" + str(linear_cnt) + "_IN_N", sub_module.in_features, config_file)
            write_define("LINEAR_" + str(linear_cnt) + "_OUT_N", sub_module.out_features, config_file)
            linear_cnt = linear_cnt + 1
            config_file.write("\n")
#         elif type(sub_module) is torch.nn.ReLU:
#             write_define("LINEAR_" + str(linear_cnt) + "_IN_N", sub_module.in_features, config_file)
    config_file.close()

In [6]:
generateConfig(model, 28, 28, 'config.h')

In [7]:
for name, sub_module in model.named_modules():
    if type(sub_module) is torch.nn.Conv2d:
        print(sub_module.weight.size())
        print(sub_module.bias.size())

torch.Size([16, 1, 3, 3])
torch.Size([16])
torch.Size([32, 16, 3, 3])
torch.Size([32])
torch.Size([32, 32, 3, 3])
torch.Size([32])


In [12]:
# import numpy np

# import numpy as np  #借助numpy模块的set_printoptions()函数，将打印上限设置为无限即可

def generate_params(model, path_to_config):

    config_file = open(path_to_config, "w")
    cov_cnt = 0
    linear_cnt = 0

    for name, sub_module in model.named_modules():
        if type(sub_module) is torch.nn.Conv2d:
            w = sub_module.weight.detach()
#             print(sub_module.weight.detach().size())
            str_h = 'float conv_' + str(cov_cnt) + '_w' + '[' + str(w.size()[0]) + ']' + '[' + str(w.size()[1]) + ']' \
            + '[' + str(w.size()[2]) + ']' + '[' + str(w.size()[3]) + '];\n' 
        
            
            config_file.write(str_h)
            
            
            b = sub_module.bias.detach()
            str_h = 'float conv_' + str(cov_cnt) + '_b' + '[' + str(b.size()[0]) + '];' + '\n' 
    
            cov_cnt = cov_cnt + 1    
            config_file.write(str_h)
        elif type(sub_module) is torch.nn.Linear:
            w = sub_module.weight.detach()
            str_h = 'float linear_' + str(linear_cnt) + '_w' + '[' + str(w.size()[0]) + ']'  \
            + '[' + str(w.size()[1]) + '];' + '\n' 
            
            config_file.write(str_h)
            
            w = sub_module.bias.detach()
            str_h = 'float linear_' + str(linear_cnt) + '_b' + '[' + str(w.size()[0]) + '];' + '\n' 
            
            config_file.write(str_h)
            linear_cnt = linear_cnt + 1
    
    config_file.close()
generate_params(model, 'params.h')

In [9]:
import numpy as np

def generate_params(model, path_to_config):
    
    cov_cnt = 0
    linear_cnt = 0

    for name, sub_module in model.named_modules():
        if type(sub_module) is torch.nn.Conv2d:
            w = sub_module.weight.detach().numpy()
            w.tofile('prama/' + 'conv_' + str(cov_cnt) + '_w.bin')
            b = sub_module.bias.detach().numpy()
            b.tofile('prama/' + 'conv_' + str(cov_cnt) + '_b.bin')
            cov_cnt = cov_cnt + 1    
        elif type(sub_module) is torch.nn.Linear:
            w = sub_module.weight.detach().numpy()
            w.tofile('prama/' + 'linear_' + str(linear_cnt) + '_w.bin')
            b = sub_module.bias.detach().numpy()
            b.tofile('prama/' + 'linear_' + str(linear_cnt) + '_b.bin')          
            linear_cnt = linear_cnt + 1

generate_params(model, 'params.h')

In [22]:
np_test = np.array([[-0.00539026, 0.0], [0.0, 0.0]], dtype=np.float32)
print(np_test)
np_test.tofile('bin/np_test.bin')
print(np_test.dtype)
a = np.fromfile('bin/np_test.bin', dtype=np.float32)
print(a)

[[-0.00539026  0.        ]
 [ 0.          0.        ]]
float32
[-0.00539026  0.          0.          0.        ]


In [24]:
for name, sub_module in model.named_modules():
    if type(sub_module) is torch.nn.Conv2d:
        b = sub_module.bias.detach().numpy()
        print(b)
  

[-0.0071761  -0.26500645 -0.13695267  0.33718866 -0.03437755  0.0664001
  0.30932966 -0.10923674 -0.1830189  -0.18321484 -0.0201841  -0.06724987
  0.25174853  0.18315771 -0.01448927  0.02068483]
[-0.07783938  0.05491597  0.06421365  0.08444484  0.05854337 -0.08153045
  0.05988433 -0.04162843 -0.02867602  0.06015589  0.10296108 -0.05653273
  0.11243331  0.07066417  0.0917825  -0.04943069  0.00214054 -0.03915513
  0.03478377  0.05991796 -0.0030085   0.07856456 -0.0235376  -0.01437902
  0.07562374  0.05689866 -0.0407959  -0.01940347  0.01574142  0.03077894
 -0.06264897  0.05679629]
[-0.00963615 -0.00520952  0.0519227   0.03503868  0.02174477  0.01664181
  0.02904369  0.06629303  0.02722135  0.00285146  0.02328476 -0.04280936
  0.01151723  0.03489932  0.04343028 -0.07362875 -0.04998298  0.05261745
 -0.07570004 -0.05914679 -0.00493663 -0.04692518 -0.05160992 -0.01051963
  0.00126685  0.05247795 -0.04425753  0.03408209 -0.02495088 -0.06547379
  0.00545672 -0.077207  ]
